## Day 7. 데이터뽀개기 중급 파이썬 스터디 발표자료

### Reference

- 아래 글들을 참고해서 작성했습니다. 감사합니다!
- [24. Context manager](https://ddanggle.gitbooks.io/interpy-kr/content/ch24-context-manager.html)
- [파이썬 context manager, with구문으로 안전한 리소스 관리를 하자.](https://velog.io/@sjquant/%ED%8C%8C%EC%9D%B4%EC%8D%AC-context-manager-with%EA%B5%AC%EB%AC%B8%EC%9C%BC%EB%A1%9C-%EC%95%88%EC%A0%84%ED%95%9C-%EB%A6%AC%EC%86%8C%EC%8A%A4-%EA%B4%80%EB%A6%AC%EB%A5%BC-%ED%95%98%EC%9E%90.)
- [텐서플로우 문서 한글 번역본](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/get_started/basic_usage.html)
- [이펙티브 텐서플로 2.0](https://tensorflow.blog/2019/02/21/%EC%9D%B4%ED%8E%99%ED%8B%B0%EB%B8%8C-%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C-2-0/)


### 7-1.  with

- 컨텍스트 매니저는 원하는 타이밍에 정확하게 리소스를 할당하고 제공하는 역할. 가장 많이 사용되는 컨텍스트 매니저는 with 문입니다. 

- with 를 사용할 때 
```
with open('some_file', 'w') as opened_file:
    opened_file.write('Hola')
```

- 위 코드와 동일한 수행을 하는 코드
```
file = open('somefile', 'w')
try:
    file.write('Hola')
finally:
    file.close()
```

- 예시와 비교해보면 간단히 with을 사용함으로써 많은 표준 코드들을 삭제했음, with 문을 사용해서 가장 큰 장점은 감싸진 블록이 존재하든지 관심을 가지지 않아도 파일은 확실히 닫아진다는 것입니다.
- 일반적으로 파일을 잠그거나 해제하거나, (이미 보여드렸듯) 열려있는 파일을 닫을 때 컨텍스트 매니저를 사용함

### 7-2. Class로 Context Manager 향상시키기

- 최소한 컨텍스트 매니저는 `__enter__` 과 `__exit__` 메소드 (Magic Methods)를 가지고 있음
- 파일을 여는 컨텍스트 매니저를 만들어보면서 기본에 대해서 살펴봅시다.

In [1]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, trace_back):
        self.file_obj.close()

In [8]:
with File('demo.txt', 'wb') as opened_file:
    opened_file.write(b'Hola!')

- `__exit__` 함수는 세 가지 전달 인자를 받습니다. 컨텍스트 매니저 class의 일부인 모든 `__exit__` 메소드에 필요함
    * 1) with문은 File(class)의 `__exit__` 메소드를 저장, 
    * 2) File(class)의 `__enter__` 메소드를 호출
    * 3) `enter` 메소드는 파일을 열고 파일을 반환
    * 4) 열려진 파일 처리는 opened_file을 통과합니다.
    * 5) .write()를 사용해서 파일을 쓸 수 있습니다.
    * 6) with문은 저장된 `__exit__` 문을 호출
    * 7) `__exit__`문은 파일을 닫습니다.

### 7-3. handling exception

- 아직까지 `__exit__` 메소드의 인자인 type, value, trace_back에 대해서는 아직 이야기를 나누지 않았음
- 4번째에서 6번째 단계 사이에서 예외가 발생한다면, 파이썬은 예외에서의 type, value, trace_back을 `__exit__`메소드에 통과시킵니다. 
- `__exit__` 메소드에서는 다음에 어떤 단계들이 요구되는 지에 따라 파일을 닫을 수 있는 방법을 결정해야합니다. 이 경우에는 거기까지는 생각하지 않겠습니다.
- 어떤 파일 객체에서 예외를 발생한다면 어떻게 될까요? 파일 객체가 지원되지 않는 메소드에 접근하려 한다고 생각해보겠습니다. 예를 들면,

In [9]:
with File('demo.txt', 'wb') as opened_file:
    opened_file.undefined_function(b'Hola!')

AttributeError: '_io.BufferedWriter' object has no attribute 'undefined_function'

- with 문이 에러를 만나면, 어떤식으로 처리되는지 단계들을 살펴보기

    * 1) type, value, traceback의 에러가 `_exit`메소드를 통과합니다.
    * 2) `__exit__` 메소드가 예외를 처리하도록 합니다.
    * 3) `__exit__` 메소드가 True 를 반환한다면, 예외는 적절하게 된 것입니다.
    * 4) `__exit__` 메소드가 True가 아닌 다른 것을 반환하면, with문에서 예외가 발생합니다.

- 위 경우에는 `__exit__` 메소드는 None(리턴 문이 어떤 것도 만나지 않는다면 None을 반환)을 반환합니다. 그래서, with문은 예외를 발생시킵니다.

In [11]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, trace_back):
        print("예외는 처리되어야 합니다")
        self.file_obj.close()
        return True

In [13]:
with File('demo.txt', 'wb') as opened_file:
    opened_file.undefined_function()

예외는 처리되어야 합니다


- `__exit__` 메소드가 True를 반환한다면, with문에서 에외는 발생하지 않을 것입니다.

### 7-4. 제너레이터를 활용해서 컨텍스트 매니저 향상시키기

- 데코레이터와 제너레이터를 활용해서 컨텍스트 매니저를 향상시킬 수도 있음 (파이썬은 contextlib 모듈) 
- class 대신에, 컨텍스트 매니저를 제너레이터 함수를 이용해서 향상시킬 수도 있습니다. 의미없는 예시를 보면서, 기본부터 확인 해보겠습니다.

In [14]:
from contextlib import contextmanager

@contextmanager
def open_file(name):
    f = open(name, 'wb')
    yield f
    f.close()

- 메소드는 제너레이터, yield, 데코레이터에 관한 지식들이 필요합니다.
-  이 예시에서는 일어날 수 있는 예외를 처리하지는 못합니다. 이전의 메소드와 동일한 방법으로 동작합니다.

    * 1. 파이썬이 yield 키워드를 만난다면, 일반적인 함수 대신에 제너레이터를 만듦
    * 2. 데코레이션이 있기 때문에 컨텍스트 매니저는 함수 이름(open_file)을 전달 인자로서 호출
    * 3. contextmanager 함수는 GeneratorContextManager로 감싸진 제너레이터를 반환
    * 4. GeneratorContextManager는 open_file함수를 할당. 그렇기 때문에 open_file 함수를 앞으로 호출하면, 사실은 GeneratorContextManager을 호출하는 것

### 7-5.  Python Context Managers and the “with” Statement (Example)


In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

class SQLAlchemyDBConnection(object):
    """SQLAlchemy database connection"""
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.session = None

    # with구문 진입시에 db와 connection을 하고
    # ORM을 사용하기 위한 session을 만들어준다.
    def __enter__(self):
        engine = create_engine(self.connection_string)
        Session = sessionmaker()
        self.session = Session(bind=engine)
        return self

    # with구문을 빠져나오기 전 session의 종료를 장한다.
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

```
conn_str = 'mssql+pydobc://server_name/db_name?driver=SQL+Server'
db = SQLAlchemyDBConnection(conn_str)
with db:
    customer = db.session.query(Customer).filter_by(id=123).one()
    print(customer.name)
```    

### 7-6. Async Context Manager

- Context Manager는 모두 동기적으로 실행됩니다. Python 3.7부터는 Context Manager를 비동기적으로 사용할 수 있음

In [18]:
# from python 3.5

class AsyncContextManager:
    async def __aenter__(self):
        await log('entering context')

    async def __aexit__(self, exc_type, exc, tb):
        await log('exiting context')

```
# from python 3.7 

from contextlib import asynccontextmanager

@asynccontextmanager
async def get_connection():
    conn = await acquire_db_connection()
    try:
        yield conn
    finally:
        await release_db_connection(conn)

async def get_all_users():
    async with get_connection() as conn:
        return conn.query('SELECT ...')
```

### 7-7. tensorflow 에서 context manager

```
sess = tf.Session()
sess.run(...)
sess.close()
```

```
with tf.Session() as sess:
  sess.run(...)
```

- 딥러닝 라이브러리 tensorflow에서는 대부분 기능들에서 컨텍스트 매니저가 포함되어 있음
- tensorflow 세션을 열고, 실행하고 리소드들이 더이상 필요하지 않을 때 해제하기 위하여 with문과 함께 컨텍스트 매니저로 실행되고 있음 (close() 메서드가 입력하지 않아도 명시적으로 실행됨)

```
c = tf.constant(...)
sess = tf.Session()
with sess.as_default():
  print(c.eval())
  
with sess.as_default():
  print(c.eval())

sess.close()  
```

- as_default 컨텍스트 매니저는 컨텍스트를 빠져나왔을 때 세션을 닫지 않으며, 명시적으로 세션을 닫아야 함

- tensorflow는 context manager를 이용해서 변수들을 그래프로 만들고, 할당하고 있는데 with 안에서 내용이 길어지다보면 너무 복잡해질 수 있음
- tesorflow에서는 즉시 실행(eager execution) 모드도 제공하고 있는데, 2.0부터는 즉시 실행 모드 (tf.keras가 eager mode로 동작하여, 디버깅/프로토타이핑을 쉽게할 수 있음)

```
import tensorflow as tf
tf.enable_eager_execution()
```